In [7]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [8]:
spark_sess= SparkSession.builder.appName("Cumulative_Windows").getOrCreate()
df_spark = spark_sess.read.csv("train.csv", header=True, inferSchema=True)
windowSpec = Window.partitionBy('Store').orderBy('Date').rowsBetween(Window.unboundedPreceding,Window.currentRow)

**Sales up to the Current Date**

In [9]:
df_spark=df_spark.withColumn('Cumulative_Sales', sum(col('Sales')).over(windowSpec))
df_spark.show(5)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+----------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|Cumulative_Sales|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+----------------+
|   28|        2|2013-01-01|    0|        0|   0|    0|           a|            1|               0|
|   28|        3|2013-01-02| 4958|      609|   1|    0|           0|            1|            4958|
|   28|        4|2013-01-03| 5287|      641|   1|    0|           0|            1|           10245|
|   28|        5|2013-01-04| 5481|      623|   1|    0|           0|            1|           15726|
|   28|        6|2013-01-05| 2070|      287|   1|    0|           0|            0|           17796|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+----------------+
only showing top 5 rows



**Average from first day up to the current row**

In [10]:
df_spark=df_spark.withColumn('Cumulative_Avg_Sales', avg(col('Sales')).over(windowSpec))
df_spark.show(10)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+----------------+--------------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|Cumulative_Sales|Cumulative_Avg_Sales|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+----------------+--------------------+
|   28|        2|2013-01-01|    0|        0|   0|    0|           a|            1|               0|                 0.0|
|   28|        3|2013-01-02| 4958|      609|   1|    0|           0|            1|            4958|              2479.0|
|   28|        4|2013-01-03| 5287|      641|   1|    0|           0|            1|           10245|              3415.0|
|   28|        5|2013-01-04| 5481|      623|   1|    0|           0|            1|           15726|              3931.5|
|   28|        6|2013-01-05| 2070|      287|   1|    0|           0|            0|           17796|              3559.2|
|   28|        7|2013-01-06|    

**Cumulative Count**

In [11]:
df_spark=df_spark.withColumn('Days_Since_Start', count(col('Sales')).over(windowSpec))
df_spark.show(10)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+----------------+--------------------+----------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|Cumulative_Sales|Cumulative_Avg_Sales|Days_Since_Start|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+----------------+--------------------+----------------+
|   28|        2|2013-01-01|    0|        0|   0|    0|           a|            1|               0|                 0.0|               1|
|   28|        3|2013-01-02| 4958|      609|   1|    0|           0|            1|            4958|              2479.0|               2|
|   28|        4|2013-01-03| 5287|      641|   1|    0|           0|            1|           10245|              3415.0|               3|
|   28|        5|2013-01-04| 5481|      623|   1|    0|           0|            1|           15726|              3931.5|               4|
|   28|        6|2013-01-05| 2070|

**Cumulative Growth Percentage**

In [17]:
df_spark=df_spark.withColumn('First_Day_Sales', first(col('Sales')).over(windowSpec)).withColumn('Cumulative_Growth_Percent', ((col('Cumulative_Sales') - col('First_Day_Sales'))/col('First_Day_Sales'))*lit(100))
df_spark.show(5)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+----------------+--------------------+----------------+---------------+-------------------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|Cumulative_Sales|Cumulative_Avg_Sales|Days_Since_Start|First_Day_Sales|Cumulative_Growth_Percent|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+----------------+--------------------+----------------+---------------+-------------------------+
|   28|        2|2013-01-01|    0|        0|   0|    0|           a|            1|               0|                 0.0|               1|              0|                     NULL|
|   28|        3|2013-01-02| 4958|      609|   1|    0|           0|            1|            4958|              2479.0|               2|              0|                     NULL|
|   28|        4|2013-01-03| 5287|      641|   1|    0|           0|            1|           10245| 

In [19]:
df_spark=df_spark.withColumn('Non_Zero_Sales', when(col('Sales')>0,col('Sales')))
df_spark.show(10)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+----------------+--------------------+----------------+---------------+-------------------------+--------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|Cumulative_Sales|Cumulative_Avg_Sales|Days_Since_Start|First_Day_Sales|Cumulative_Growth_Percent|Non_Zero_Sales|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+----------------+--------------------+----------------+---------------+-------------------------+--------------+
|   28|        2|2013-01-01|    0|        0|   0|    0|           a|            1|               0|                 0.0|               1|              0|                     NULL|          NULL|
|   28|        3|2013-01-02| 4958|      609|   1|    0|           0|            1|            4958|              2479.0|               2|              0|                     NULL|          4958|
|   28|        4|2013-01-

In [24]:
df_spark=df_spark.withColumn('First_Non_Zero_Sales', first('Non_Zero_Sales', ignorenulls=True).over(windowSpec))
df_spark.show(10)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+----------------+--------------------+----------------+---------------+-------------------------+--------------+--------------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|Cumulative_Sales|Cumulative_Avg_Sales|Days_Since_Start|First_Day_Sales|Cumulative_Growth_Percent|Non_Zero_Sales|First_Non_Zero_Sales|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+----------------+--------------------+----------------+---------------+-------------------------+--------------+--------------------+
|   28|        2|2013-01-01|    0|        0|   0|    0|           a|            1|               0|                 0.0|               1|              0|                     NULL|          NULL|                NULL|
|   28|        3|2013-01-02| 4958|      609|   1|    0|           0|            1|            4958|              2479.0|               2

In [ ]:
df_spark=df_spark.withColumn('Cumulative_Non_Zero_Growth_Percent', ((col('Cumulative_Sales') - col('First_Non_Zero_Sales'))/col('First_Non_Zero_Sales'))*lit(100))
df_spark.show(10)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+----------------+--------------------+----------------+---------------+-------------------------+--------------+--------------------+----------------------------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|Cumulative_Sales|Cumulative_Avg_Sales|Days_Since_Start|First_Day_Sales|Cumulative_Growth_Percent|Non_Zero_Sales|First_Non_Zero_Sales|Cumulative_Non_Zero_Growth_Percent|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+----------------+--------------------+----------------+---------------+-------------------------+--------------+--------------------+----------------------------------+
|   28|        2|2013-01-01|    0|        0|   0|    0|           a|            1|               0|                 0.0|               1|              0|                     NULL|          NULL|                NULL|                              NU

25/11/25 01:54:03 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1086721 ms exceeds timeout 120000 ms
25/11/25 01:54:03 WARN SparkContext: Killing executors is not supported by current scheduler.
25/11/25 01:54:03 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 